# Transform Non-Image Data into OMOP CDM

This part can be skipped if your non-imaging data already exist in OMOP CDM. In this case, you would need to go to the `Part II: Tranform Image Data` notebook.

### Prerequisites
* Download ODBC Driver 18 from web <https://learn.microsoft.com/en-us/sql/connect/odbc/linux-mac/install-microsoft-odbc-driver-sql-server-macos?view=sql-server-ver16>
* OMOP CDM instance with the connection string and authentication information. If your database access limits IP addresses, make sure to add your IP address before running the connection strings.
* Install SQL processing package, i.e., pyodbc.
* Files or database that non-image data is stored: demographic, clinical assessments, labs, visits, etc.

### Patient demographic data
- Downloaded from ADNI demographic file (Subject Demographics) and saved in my local folder.
- This will be transformed and loaded to the Person table

In [21]:
import pandas as pd
# Load the file
patient_demo = pd.read_csv('./files/ADNI/PTDEMOG_28Mar2024.csv')
# Subbset required fields from the file
patient_demo_staging = patient_demo[['PTID', 'PTGENDER', 'PTDOB', 'PTDOBYY', 'PTRACCAT', 'PTETHCAT']].copy() 

In [ ]:
# Standard omop gender concepts includes female/male
patient_demo_staging = patient_demo_staging[patient_demo_staging['PTGENDER'].isin([1,2])]

# Assign an integer value for each unique PatientID
patient_demo_staging['person_id'], _ = pd.factorize(patient_demo_staging['PTID'])
patient_demo_staging['person_id'] = patient_demo_staging['person_id'] + 1 

# Codify Gender - replace 'F' with 8532 and 'M' with 8507
gender_codification = {2: 8532, 1: 8507}
patient_demo_staging['gender_concept_id'] = patient_demo_staging['PTGENDER'].replace(gender_codification)

# birth year and month
patient_demo_staging['year_of_birth'] = patient_demo_staging['PTDOBYY']
patient_demo_staging['month_of_birth'] = patient_demo_staging['PTDOB'].str.slice(0,2).astype(int)

# race category
race_codification = {1:8657, 2:8515, 3: 8557, 4:8516, 5:8527}
patient_demo_staging['race_concept_id'] = patient_demo_staging['PTRACCAT'].map(lambda x: race_codification.get(x, 0)).astype(int)

# ethnicity
ethnicity_codification = {1:38003563, 2:38003564}
patient_demo_staging['ethnicity_concept_id'] = patient_demo_staging['PTETHCAT'].map(lambda x: ethnicity_codification.get(x, 0)).astype(int)

# drop duplicate if applicable
patient_demo_staging = patient_demo_staging.drop_duplicates(subset='PTID', keep = 'first')

# source name
patient_demo_staging['source'] = 'ADNI'

#patient_demo_staging.head()

In [ ]:
import psycopg2

conn = psycopg2.connect(
    database="",
    user="",
    password="",
    host="",
    port="",
    connect_timeout = 6000
)

cursor = conn.cursor()

In [25]:
# Update PERSON
cursor = conn.cursor()

# Update PERSON
sql = '''
    INSERT INTO adni.person (person_id, gender_concept_id, year_of_birth, month_of_birth, race_concept_id, ethnicity_concept_id, gender_source_value) 
    VALUES (%s,%s,%s,%s,%s,%s,%s)
    '''
for index, row in patient_demo_staging.iterrows():
    cursor.execute(sql, (row['person_id'], row['gender_concept_id'], row['year_of_birth'], row['month_of_birth'], row['race_concept_id'], row['ethnicity_concept_id'], row['PTGENDER']))

conn.commit()

In [39]:
# Create the registry_idmap table if it doesn't exist
# This will keep track of which OMOP Person_id belongs to ADNI PTID

ddl_statement = """
CREATE TABLE adni.registry_idmap(
    source_id varchar(250) NOT NULL,
    person_id integer NOT NULL,
    source_name varchar(250)
);
"""

cursor.execute(ddl_statement)
conn.commit()

sql = '''
    INSERT INTO adni.registry_idmap (source_id, person_id, source_name) 
    VALUES (%s,%s,%s)
    '''
for index, row in patient_demo_staging.iterrows():
    cursor.execute(sql, (row['PTID'], row['person_id'], row['source']))

conn.commit()

### NPI-Q and NPI Scores

In [27]:
import pandas as pd

# Load the file
npi = pd.read_csv('./files/ADNI/NPI_10Apr2024.csv')
# Subbset required fields from the file
# This should be selected by the researcher(s)
npi_staging = npi[['PTID', 'Phase', 'VISCODE', 'VISCODE2', 'VISDATE', 'EXAMDATE', 'NPIATOT',
                                     'NPIBTOT', 'NPICTOT', 'NPIDTOT', 'NPIETOT', 'NPIFTOT', 'NPIGTOT', 'NPIHTOT',
                                     'NPIITOT', 'NPIJTOT', 'NPIKTOT', 'NPILTOT', 'NPITOTAL']].copy()

In [ ]:
# transform the table from wide to long
# add custom concept for NPI as SNOMED only exists for NPI-Q
# add custom concept for each questions
# npi_staging.head()

In [ ]:
# Define the columns to keep (not starting with 'NPI')
cols_to_keep = [col for col in npi_staging.columns if not col.startswith('NPI')]

# Define the columns that need to be transformed
npi_columns = [col for col in npi_staging.columns if col.startswith('NPI')]

# Melt the DataFrame
npi_long = npi_staging.melt(id_vars=cols_to_keep, value_vars=npi_columns,
                    var_name='NPI_section', value_name='NPI_value')

# Extract the first letter after 'NPI' for the 'NPI_section'
npi_long['NPI_section'] = npi_long['NPI_section'].apply(lambda x: x[3])

# npi_long.head()

In [30]:
npi_long.shape #npi_long.shape

(90324, 8)

In [31]:
npi_long = npi_long.merge(patient_demo_staging[['source_id', 'person_id']], left_on= 'PTID', right_on = 'source_id', how ='left')

In [20]:
41571200 + 2000000000 #Assessment scales (parent node of NPI-Q) + custom concept id convention

2041571200

In [21]:
# Update CONCEPT for NPI
sql = '''
    INSERT INTO dbo.concept (concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,concept_code,valid_start_date,valid_end_date) 
    VALUES 
    (2041571200, 'Neuropsychiatric Inventory', 'Measurement', '0', '0', '0', '1994-01-01', '2099-12-31'),
    (2041571201, 'Neuropsychiatric Inventory Section A. Delusions: Item score', 'Measurement', '0', '0', '0', '1994-01-01', '2099-12-31'),
    (2041571202, 'Neuropsychiatric Inventory Section B. Hallucinations: Item score', 'Measurement', '0', '0', '0', '1994-01-01', '2099-12-31'),
    (2041571203, 'Neuropsychiatric Inventory Section C. Agitation/Aggression: Item score', 'Measurement', '0', '0', '0', '1994-01-01', '2099-12-31'),
    (2041571204, 'Neuropsychiatric Inventory Section D. Depression/Dysphoria: Item score', 'Measurement', '0', '0', '0', '1994-01-01', '2099-12-31'),
    (2041571205, 'Neuropsychiatric Inventory Section E. Anxiety: Item score', 'Measurement', '0', '0', '0', '1994-01-01', '2099-12-31'),
    (2041571206, 'Neuropsychiatric Inventory Section F. Elation/Euphoria: Item score', 'Measurement', '0', '0', '0', '1994-01-01', '2099-12-31'),
    (2041571207, 'Neuropsychiatric Inventory Section G. Apathy/Indifference: Item score', 'Measurement', '0', '0', '0', '1994-01-01', '2099-12-31'),
    (2041571208, 'Neuropsychiatric Inventory Section H. Disinhibition: Item score', 'Measurement', '0', '0', '0', '1994-01-01', '2099-12-31'),
    (2041571209, 'Neuropsychiatric Inventory Section I. Irritability/Lability: Item score', 'Measurement', '0', '0', '0', '1994-01-01', '2099-12-31'),
    (2041571210, 'Neuropsychiatric Inventory Section J. Aberrant Motor Behavior: Item score', 'Measurement', '0', '0', '0', '1994-01-01', '2099-12-31'),
    (2041571211, 'Neuropsychiatric Inventory Section K. Sleep: Item score', 'Measurement', '0', '0', '0', '1994-01-01', '2099-12-31'),
    (2041571212, 'Neuropsychiatric Inventory Section L. Appetite and eating disorders: Item score', 'Measurement', '0', '0', '0', '1994-01-01', '2099-12-31'),
    (2041571213, 'Neuropsychiatric Inventory Total Score', 'Measurement', '0', '0', '0', '1994-01-01', '2099-12-31')
    '''
cursor.execute(sql)
conn.commit()

In [22]:
# Update CONCEPT for NPI-Q
sql = '''
    INSERT INTO dbo.concept (concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,concept_code,valid_start_date,valid_end_date) 
    VALUES 
    (2041571214, 'Neuropsychiatric Inventory Q Delusions', 'Measurement', '0', '0', '0', '1994-01-01', '2099-12-31'),
    (2041571215, 'Neuropsychiatric Inventory Q Hallucinations', 'Measurement', '0', '0', '0', '1994-01-01', '2099-12-31'),
    (2041571216, 'Neuropsychiatric Inventory Q Agitation/Aggression', 'Measurement', '0', '0', '0', '1994-01-01', '2099-12-31'),
    (2041571217, 'Neuropsychiatric Inventory Q Depression/Dysphoria', 'Measurement', '0', '0', '0', '1994-01-01', '2099-12-31'),
    (2041571218, 'Neuropsychiatric Inventory Q Anxiety', 'Measurement', '0', '0', '0', '1994-01-01', '2099-12-31'),
    (2041571219, 'Neuropsychiatric Inventory Q Elation/Euphoria', 'Measurement', '0', '0', '0', '1994-01-01', '2099-12-31'),
    (2041571220, 'Neuropsychiatric Inventory Q Apathy/Indifference', 'Measurement', '0', '0', '0', '1994-01-01', '2099-12-31'),
    (2041571221, 'Neuropsychiatric Inventory Q Disinhibition', 'Measurement', '0', '0', '0', '1994-01-01', '2099-12-31'),
    (2041571222, 'Neuropsychiatric Inventory Q Irritability/Lability', 'Measurement', '0', '0', '0', '1994-01-01', '2099-12-31'),
    (2041571223, 'Neuropsychiatric Inventory Q Aberrant Motor Behavior', 'Measurement', '0', '0', '0', '1994-01-01', '2099-12-31'),
    (2041571224, 'Neuropsychiatric Inventory Q Sleep', 'Measurement', '0', '0', '0', '1994-01-01', '2099-12-31'),
    (2041571225, 'Neuropsychiatric Inventory Q Appetite and eating disorders', 'Measurement', '0', '0', '0', '1994-01-01', '2099-12-31')
    '''
cursor.execute(sql)
conn.commit()

In [ ]:
# Import measurement table
sql_query = "SELECT max(measurement_id) FROM dbo.measurement"
pd.read_sql_query(sql_query, conn)
#560169

In [32]:
npi_long.NPI_value.unique()

array([ 0.,  1.,  3., nan,  2.,  8.,  4.,  6.,  9., 12., 19., 20., 10.,
       14., 17.,  5.,  7., 13., 16., 18., 23., 26., 32., 28., 47., 29.,
       11., 22., 24., 15., 37., 25., 27., 34., 21., 31., 33., 46., 43.,
       35., 30., 39., 36., 59., 38., 60., 53., 51., 49., 41., 52., 40.,
       79., 44., 88., 45., 65., 67., 61., 55., 57., 48., 58., 78., 56.,
       42., 54., 71.])

In [119]:
# Drop rows with NPI_value NAN
def is_numeric(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

npi_long_new = npi_long.dropna(subset=['NPI_value']).copy()
npi_long_new['NPI_value'] = npi_long_new['NPI_value'].apply(lambda x: round(float(x), 6) if is_numeric(x) else x)
npi_long_new.shape #(89659, 10)

(89659, 10)

In [120]:
# Assign custom concepts per NPI question section
npi_codes = {
    'A': 2041571201,
    'B': 2041571202,
    'C': 2041571203,
    'D': 2041571204,
    'E': 2041571205,
    'F': 2041571206,
    'G': 2041571207,
    'H': 2041571208,
    'I': 2041571209,
    'J': 2041571210,
    'K': 2041571211,
    'L': 2041571212,
    'T': 2041571213
}

npi_long_new['concept_id'] = npi_long_new['NPI_section'].map(npi_codes)

In [121]:
# Create measurement_id 
# max measurement_id in the db as of 5/17/2024 = 560169
npi_long_new['measurement_id'] = range(560169, 560169 + npi_long_new.shape[0])

In [123]:
npi_long_new['VISDATE'] = pd.to_datetime(npi_long_new['VISDATE'].str.strip(), errors='coerce')

In [124]:
# Convert Exam date and visit date
npi_long_new['EXAMDATE_DT'] = pd.to_datetime(npi_long_new['EXAMDATE'].str.strip(), errors = 'coerce')

In [ ]:
# Look through NA exam dates
npi_long_new[npi_long_new['EXAMDATE_DT'].isna()]

In [ ]:
# Look through NA visit dates
npi_long_new[npi_long_new['VISDATE'].isna()]

In [127]:
# To maximize retaining data, we will use visit date as measurement date
# Drop rows where visit date is NAN
npi_long_new = npi_long_new.dropna(subset=['VISDATE'])
npi_long_new.shape #(89568, 13)

(89568, 13)

In [129]:
sql = '''
    INSERT INTO dbo.measurement (measurement_id, person_id, measurement_concept_id, measurement_date, measurement_type_concept_id, value_as_number, measurement_source_concept_id) 
    VALUES (%s,%s,%s,%s,%s,%s,%s)
'''

# Define batch size
batch_size = 20000
batch = []

# Iterate through the DataFrame
for index, row in npi_long_new.iterrows():
    # Append row data to the batch
    batch.append((row['measurement_id'], row['person_id'], row['concept_id'], row['VISDATE'], 32817, row['NPI_value'], row['concept_id']))

    # Check if batch size is reached or if it's the last row
    if len(batch) == batch_size or (index == len(npi_long_new) - 1):
        # Execute the batch
        cursor.executemany(sql, batch)
        conn.commit()  # Commit after each batch
        batch = []  # Clear the batch for the next set of rows

# Close the cursor and connection
cursor.close()
conn.close()

In [ ]:
# check if the measurement table is updated with NPI values

sql_query = "SELECT * FROM dbo.measurement where measurement_source_concept_id between 2041571201 and 2041571213"
df_measurement_npi = pd.read_sql_query(sql_query, conn)

In [8]:
agg_npi = df_measurement_npi.groupby('person_id')['measurement_date'].agg(['min', 'max']).reset_index()

### Update the Observation_period table

In [15]:
import pandas as pd

sql = 'select * from dbo.observation_period'
df_obs_period = pd.read_sql_query(sql, conn)

/var/folders/wh/p78ddrtn39j879rx8h8fs8fr0000gn/T/ipykernel_91043/3309558774.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_obs_period = pd.read_sql_query(sql, conn)


In [ ]:
df_obs_period_npi = df_obs_period.merge(agg_npi, on = 'person_id')
df_obs_period_npi['older_date'] = df_obs_period_npi[['observation_period_start_date', 'min']].min(axis = 1)
df_obs_period_npi['later_date'] = df_obs_period_npi[['observation_period_end_date', 'max']].max(axis = 1)

In [14]:
sql_query = '''
    UPDATE dbo.observation_period
    SET observation_period_start_date = %s, observation_period_end_date = %s
    WHERE person_id = %s
'''

for index, row in df_obs_period_npi.iterrows():
    cursor.execute(sql_query, (row['older_date'], row['later_date'], row['person_id']))

conn.commit()

In [18]:
cursor.close()
conn.close()

## Ingesting diagnosis codes

In [6]:
import pandas as pd

dx = pd.read_csv('./files/ADNI/DXSUM_diagnosis.csv')

In [ ]:
dx = dx.merge(patient_demo_staging[["source_id", "person_id"]], left_on = 'PTID', right_on = 'source_id', how = 'left')
dx.head()

In [17]:
dx_no_control = dx[dx['DIAGNOSIS']>1].copy().reset_index()
dx_no_control['DIAGNOSIS'].unique()

array([3., 2.])

In [18]:
dx_no_control.shape, dx.shape

((8677, 6), (13842, 5))

In [13]:
mapping = {2: 4297400, 3: 4182210}

In [19]:
dx_no_control['id'] = range(1,len(dx_no_control)+1)
dx_no_control['concept_id'] = dx_no_control['DIAGNOSIS'].map(mapping)
dx_no_control['condition_type_concept_id'] = 32817

In [21]:
dx_no_control.to_csv('./files/OMOP CDM Staging/dx_for_omop.csv')

In [ ]:
# Update CONDITION_OCCURRENCE
sql = '''
    INSERT INTO dbo.condition_occurrence (condition_occurrence_id, person_id, condition_concept_id, condition_start_date, condition_type_concept_id) 
    VALUES (%s,%s,%s,%s,%s)
    '''
for index, row in dx_no_control.iterrows():
    cursor.execute(sql, (row['id'], row['person_id'], row['concept_id'], row['EXAMDATE'], row['condition_type_concept_id']))

conn.commit()

### Mini-Mental Evaluation scores

In [ ]:
import pandas as pd
mmse = pd.read_csv('./files/ADNI/MMSE_08Aug2024.csv')
mmse.head()

In [5]:
mmse.columns

Index(['PHASE', 'PTID', 'RID', 'VISCODE', 'VISCODE2', 'VISDATE', 'DONE',
       'NDREASON', 'SOURCE', 'MMDATE', 'MMYEAR', 'MMMONTH', 'MMDAY',
       'MMSEASON', 'MMHOSPIT', 'MMFLOOR', 'MMCITY', 'MMAREA', 'MMSTATE',
       'WORDLIST', 'WORD1', 'WORD2', 'WORD3', 'MMTRIALS', 'MMD', 'MML', 'MMR',
       'MMO', 'MMW', 'MMLTR1', 'MMLTR2', 'MMLTR3', 'MMLTR4', 'MMLTR5',
       'MMLTR6', 'MMLTR7', 'WORLDSCORE', 'WORD1DL', 'WORD2DL', 'WORD3DL',
       'MMWATCH', 'MMPENCIL', 'MMREPEAT', 'MMHAND', 'MMFOLD', 'MMONFLR',
       'MMREAD', 'MMWRITE', 'MMDRAW', 'MMSCORE', 'ID', 'SITEID', 'USERDATE',
       'USERDATE2', 'DD_CRF_VERSION_LABEL', 'LANGUAGE_CODE', 'HAS_QC_ERROR',
       'update_stamp'],
      dtype='object')

In [6]:
data_dic = pd.read_csv('./files/DATADIC_08Aug2024.csv')
data_dic.head()

,PHASE,CRFNAME,TBLNAME,FLDNAME,TEXT,TYPE,LENGTH,DD_CRF_VERSION,CODE,UNITS,STATUS,CODE_CHANGES,MAPPING_NOTES
0,ADNI1,ADAS-Cognitive Behavior,ADAS,PTID,Participant ID,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ADNI1,ADAS-Cognitive Behavior,ADAS,RID,Participant roster ID,N,38 digits,NaN,NaN,NaN,NaN,NaN,NaN
2,ADNI1,ADAS-Cognitive Behavior,ADAS,VISCODE,Visit code,T,20 characters,NaN,NaN,NaN,NaN,NaN,NaN
3,ADNI1,ADAS-Cognitive Behavior,ADAS,EXAMDATE,Examination Date,D,10,NaN,NaN,NaN,NaN,NaN,NaN
4,ADNI1,ADAS-Cognitive Behavior,ADAS,VISDATE,Assessment EXAMDATE when present; otherwise Re...,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
data_dic[data_dic['TBLNAME']=="MMSE"]['FLDNAME'].unique()

array(['PTID', 'RID', 'VISCODE', 'EXAMDATE', 'VISDATE', 'MMDATE',
       'MMDATECM', 'MMYEAR', 'MMYEARCM', 'MMMONTH', 'MMMNTHCM', 'MMDAY',
       'MMDAYCM', 'MMSEASON', 'MMSESNCM', 'MMHOSPIT', 'MMHOSPCM',
       'MMFLOOR', 'MMFLRCM', 'MMCITY', 'MMCITYCM', 'MMAREA', 'MMAREACM',
       'MMSTATE', 'MMSTCM', 'MMBALL', 'MMFLAG', 'MMTREE', 'MMTRIALS',
       'MMD', 'MMDLTR', 'MML', 'MMLLTR', 'MMR', 'MMRLTR', 'MMO', 'MMOLTR',
       'MMW', 'MMWLTR', 'MMBALLDL', 'MMFLAGDL', 'MMTREEDL', 'MMWATCH',
       'MMPENCIL', 'MMREPEAT', 'MMHAND', 'MMFOLD', 'MMONFLR', 'MMREAD',
       'MMWRITE', 'MMDRAW', 'MMSCORE', 'ID', 'SITEID', 'USERDATE',
       'USERDATE2', 'VISCODE2', 'MMRECALL', 'MMLTR1', 'MMLTR2', 'MMLTR3',
       'MMLTR4', 'MMLTR5', 'MMLTR6', 'MMLTR7', 'WORLDSCORE', 'DONE',
       'NDREASON', 'WORDLIST', 'WORD1', 'WORD2', 'WORD3', 'WORD1DL',
       'WORD2DL', 'WORD3DL', 'DATE', 'SOURCE', 'DD_CRF_VERSION_LABEL',
       'LANGUAGE_CODE', 'HAS_QC_ERROR'], dtype=object)